In [40]:
import json
import os
import pandas as pd


def get_json_files():
    json_files = []
    for file in os.listdir("rawdata"):
        if file.endswith(".json"):
            json_files.append(file)
    return json_files


all_files = get_json_files()

In [41]:
data = pd.read_json("rawdata/" + all_files[1])

In [42]:
data.head()

,title,image_source,price,description,product_url
0,Get Well Sunshine Gift Hamper <br/>(Get Well S...,https://www.gifthampersuk.co.uk/image/cache/ca...,£50.00,Hamper ContentsAll Around The Fire Hot Chocola...,https://www.gifthampersuk.co.uk/care-hampers/g...
1,Recovery Treats Gift Hamper <br/>(Get Well Soo...,https://www.gifthampersuk.co.uk/image/cache/ca...,£45.00,Hamper Contents\r\nAlchemy Tea Rest and Recupe...,https://www.gifthampersuk.co.uk/care-hampers/r...
2,To Perk You Up Gift Hamper <br/>(Get Well Soon...,https://www.gifthampersuk.co.uk/image/cache/ca...,£43.00,Hamper Contents\r\nAnna’s Cappuccino Thins (15...,https://www.gifthampersuk.co.uk/care-hampers/a...
3,A Little Bit Of Calm Gift Hamper <br/>(Get Wel...,https://www.gifthampersuk.co.uk/image/cache/ca...,£60.00,Hamper Contents\r\nAlchemy Tea Howick Earl Gre...,https://www.gifthampersuk.co.uk/care-hampers/a...
4,Coffee Connoisseur's Dream Gift Hamper <br/>(G...,https://www.gifthampersuk.co.uk/image/cache/ca...,£50.00,Hamper ContentsGIFT ITEM: Life & Soul Cafetier...,https://www.gifthampersuk.co.uk/care-hampers/c...


In [43]:
from main import get_soup
from individualScraper import individual_product_info, dump_product_info

product_data_list_ = []
for index, row in data.iterrows():
    url = row["product_url"]
    soup = get_soup(url)
    product_info = individual_product_info(soup, url)
    product_data_list_.append(product_info)
    print(f"Scraped {index} of {len(data)}")

csv_file_name = "product_data.csv"
dump_product_info(product_data_list_, csv_file_name)

TypeError: individual_product_info() takes 1 positional argument but 2 were given